# Problem 5.1 Read in Data

In [87]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [88]:
import glob
files = glob.glob('/content/drive/MyDrive/QIS/lab5*.xlsx')
files.sort()
files
files_is = files[:8]
files_oos = files [8:]
new_df = pd.DataFrame()
for filename in files_is:
  raw = pd.read_excel(filename, index_col = None, header =0)
  raw.columns = ['name','ticker', 'exchange', 'close', 'size', 'Beme', 'eP', 'm12M', 'yr1Ret', 'ind']
  raw['year'] = int(filename[37:41])
  new_df = pd.concat([new_df, raw], ignore_index = True)

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [89]:
new_df = new_df.dropna()
new_df.shape

(37844, 11)

# Problem 5.2

In [90]:
def is_out(x):
    return abs(x - x.mean()) > x.std() * 3

In [91]:
out = new_df[['size','Beme','eP','m12M']]

#creating boolean df
outA = out.apply(is_out)

print('Outlier Boolean DF')
print(outA.head(2))
print('-------------')

print('Total Outliers')
print(outA.apply(sum))
print('-------------')
#creating boolean index
outB = outA.any(axis = 1)

print('Boolean Index')
print(outB.head(2))
print('-------------')
#remove all outliers
df = new_df[~outB]

print("Before and After Shapes")
print(new_df.shape)
print(df.shape)

#reseting index
df.reset_index(inplace = True, drop = True)
df.head(3)

Outlier Boolean DF
    size   Beme     eP   m12M
0  False  False  False  False
1  False  False  False  False
-------------
Total Outliers
size    563
Beme      3
eP       40
m12M     41
dtype: int64
-------------
Boolean Index
0    False
1    False
dtype: bool
-------------
Before and After Shapes
(37844, 11)
(37198, 11)


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,28.625,331195457.0,0.042162,0.022707,1.057143,-0.565415,43.0,2000
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,4.125,264772047.0,0.474933,-0.276364,-0.761905,2.781818,43.0,2000
2,"1st Independence Financial Group, Inc.",FIFG,NASDAQ,14.188,21493046.0,1.152143,0.053566,-0.001561,-0.184455,45.0,2000


In [92]:
df.shape

(37198, 11)

In [93]:
df.head()

,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,28.62500,331195457.0,0.042162,0.022707,1.057143,-0.565415,43.0,2000
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,4.12500,264772047.0,0.474933,-0.276364,-0.761905,2.781818,43.0,2000
2,"1st Independence Financial Group, Inc.",FIFG,NASDAQ,14.18800,21493046.0,1.152143,0.053566,-0.001561,-0.184455,45.0,2000
3,1st Source Corporation,SRCE,NASDAQ,15.80091,359974602.0,0.751640,0.102032,-0.241881,0.211745,45.0,2000
4,"1st State Bancorp, Inc.",FSBC,NASDAQ,18.43800,60653773.0,0.994083,0.071591,-0.063291,0.084717,45.0,2000


In [94]:
data_f = df[(df['eP'] >0) &(df['close'] >5)& (df['size'] > 100000000)&(df['Beme'] >0)]
data_f.shape

(20770, 11)

In [95]:
data_f['lnSize'] = np.log(df['size'])
data_f.head()

<ipython-input-95-1b1f7ca262fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f['lnSize'] = np.log(df['size'])


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,28.62500,3.311955e+08,0.042162,0.022707,1.057143,-0.565415,43.0,2000,19.618219
3,1st Source Corporation,SRCE,NASDAQ,15.80091,3.599746e+08,0.751640,0.102032,-0.241881,0.211745,45.0,2000,19.701544
6,21st Century Insurance Group,TW,"NEW YORK STOCK EXCHANGE, INC.",14.25000,1.213328e+09,0.593874,0.014737,-0.164329,0.389856,34.0,2000,20.916633
8,3Com Corporation,COMS,NASDAQ,8.50000,2.892038e+09,1.267800,0.028235,0.520983,-0.249412,37.0,2000,21.785227
10,3D Systems Corporation,DDD,"NEW YORK STOCK EXCHANGE, INC.",6.06250,1.462983e+08,0.490755,0.048660,0.806452,0.208248,36.0,2000,18.801158


In [96]:
from scipy.stats import zscore #scipy.stats has a zscore function
import scipy.stats as stats
for i in ['Beme','eP', 'm12M', 'lnSize']:
  data_f['z' + i] = data_f.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))

<ipython-input-96-5754b203940d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f['z' + i] = data_f.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))


In [97]:
data_f.head()

,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize,zBeme,zeP,zm12M,zlnSize
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,28.62500,3.311955e+08,0.042162,0.022707,1.057143,-0.565415,43.0,2000,19.618219,-1.053904,-1.088370,2.007630,-0.914650
3,1st Source Corporation,SRCE,NASDAQ,15.80091,3.599746e+08,0.751640,0.102032,-0.241881,0.211745,45.0,2000,19.701544,0.604810,0.778249,-0.857951,-0.435663
6,21st Century Insurance Group,TW,"NEW YORK STOCK EXCHANGE, INC.",14.25000,1.213328e+09,0.593874,0.014737,-0.164329,0.389856,34.0,2000,20.916633,0.320582,-0.563931,-0.532632,0.092869
8,3Com Corporation,COMS,NASDAQ,8.50000,2.892038e+09,1.267800,0.028235,0.520983,-0.249412,37.0,2000,21.785227,1.763786,-0.284309,0.306554,0.471956
10,3D Systems Corporation,DDD,"NEW YORK STOCK EXCHANGE, INC.",6.06250,1.462983e+08,0.490755,0.048660,0.806452,0.208248,36.0,2000,18.801158,0.640134,0.152525,0.619816,-1.411251


In [98]:
import statsmodels.formula.api as smf
coefficients = pd.DataFrame()
for i in data_f['year'].unique():
  results = smf.ols(formula = 'yr1Ret~ + zBeme+ zm12M + zeP + zlnSize', data = data_f[data_f['year'] == i]).fit()
  coeff = pd.DataFrame(results.params)
  coeff.columns = [i]
  coefficients = pd.concat([coefficients, coeff], axis = 1)

In [99]:
coef = coefficients
coef = coef.transpose()
coef.head()

,Intercept,zBeme,zm12M,zeP,zlnSize
2000,0.111342,0.047378,-0.039494,0.050971,-0.084575
2001,-0.087116,0.025334,0.042380,0.008835,-0.025619
2002,0.416027,0.058798,0.014999,0.033630,-0.016940
2003,0.212177,0.019186,0.005633,0.035612,-0.003581
2004,0.100567,0.019903,0.013706,0.003512,0.018821


#Problem 5.3

In [100]:
coef.head()

,Intercept,zBeme,zm12M,zeP,zlnSize
2000,0.111342,0.047378,-0.039494,0.050971,-0.084575
2001,-0.087116,0.025334,0.042380,0.008835,-0.025619
2002,0.416027,0.058798,0.014999,0.033630,-0.016940
2003,0.212177,0.019186,0.005633,0.035612,-0.003581
2004,0.100567,0.019903,0.013706,0.003512,0.018821


In [101]:
import math
coef.loc["FM Avg"] = coef.mean()
coef.loc["Std"] = coef.std()
coef.loc['t stat'] = (coef.loc['FM Avg',:]/coef.loc['Std',:])*math.sqrt(8)
coef.tail()

,Intercept,zBeme,zm12M,zeP,zlnSize
2006,0.042725,-0.026665,0.012021,-0.011889,0.025302
2007,-0.351001,-0.002419,-0.029388,-0.009093,-0.011727
FM Avg,0.078902,0.020959,0.001469,0.014959,-0.013149
Std,0.211601,0.024987,0.024640,0.021175,0.031434
t stat,1.054661,2.372518,0.168648,1.998068,-1.183139


# Problem 5.4

In [102]:
coef.tail()

,Intercept,zBeme,zm12M,zeP,zlnSize
2006,0.042725,-0.026665,0.012021,-0.011889,0.025302
2007,-0.351001,-0.002419,-0.029388,-0.009093,-0.011727
FM Avg,0.078902,0.020959,0.001469,0.014959,-0.013149
Std,0.211601,0.024987,0.024640,0.021175,0.031434
t stat,1.054661,2.372518,0.168648,1.998068,-1.183139


As we can see the T-Stat for zBeme and zeP are both above 1.5. And so for the following we will be working with those factors.


# Problem 5.5


In [103]:
new_df2 = pd.DataFrame()
for filename in files_oos:
  raw = pd.read_excel(filename, index_col = None, header =0)
  raw.columns = ['name','ticker', 'exchange', 'close', 'size', 'Beme', 'eP', 'm12M', 'yr1Ret', 'ind']
  raw['year'] = int(filename[37:41])
  new_df2 = pd.concat([new_df2, raw], ignore_index = True)

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [104]:
new_df2.shape

(17740, 11)

In [105]:
new_df2.head()

,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,NaN,NaN,NaN,NaN,0.000000,0.000000,43.0,2008
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,3.8200,243373888.0,0.907906,0.086387,-0.643744,-0.306283,43.0,2008
2,"1st Century Bancshrs, Inc.",FCTY,NASDAQ,4.1500,40558680.0,1.406554,0.057831,-0.288136,-0.168675,45.0,2008
3,1st Constitution Bancorp,FCCY,NASDAQ,8.3093,40364929.0,1.150591,0.076970,-0.448986,-0.350248,45.0,2008
4,"1st Independence Financial Group, Inc.",FIFG,NASDAQ,NaN,NaN,NaN,NaN,0.450094,0.000000,45.0,2008


In [106]:
new_df2 = new_df2.dropna()
new_df2.shape

(9433, 11)

In [107]:
out = new_df2[['size','Beme','eP','m12M']]

#creating boolean df
outA = out.apply(is_out)

print('Outlier Boolean DF')
print(outA.head(2))
print('-------------')

print('Total Outliers')
print(outA.apply(sum))
print('-------------')
#creating boolean index
outB = outA.any(axis = 1)

print('Boolean Index')
print(outB.head(2))
print('-------------')
#remove all outliers
df2 = new_df2[~outB]

print("Before and After Shapes")
print(new_df2.shape)
print(df2.shape)

#reseting index
df2.reset_index(inplace = True, drop = True)
df2.head(3)

Outlier Boolean DF
    size   Beme     eP   m12M
1  False  False  False  False
2  False  False  False  False
-------------
Total Outliers
size    146
Beme      5
eP        6
m12M     76
dtype: int64
-------------
Boolean Index
1    False
2    False
dtype: bool
-------------
Before and After Shapes
(9433, 11)
(9202, 11)


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,3.8200,243373888.0,0.907906,0.086387,-0.643744,-0.306283,43.0,2008
1,"1st Century Bancshrs, Inc.",FCTY,NASDAQ,4.1500,40558680.0,1.406554,0.057831,-0.288136,-0.168675,45.0,2008
2,1st Constitution Bancorp,FCCY,NASDAQ,8.3093,40364929.0,1.150591,0.076970,-0.448986,-0.350248,45.0,2008


In [108]:
data_f2 = df2[(df2['eP'] >0) &(df2['close'] >5)& (df2['size'] > 100000000)&(df2['Beme'] >0)]
data_f2.shape

(4299, 11)

In [109]:
data_f2['lnSize'] = np.log(df2['size'])
data_f2.head()

<ipython-input-109-9c2ed5179e06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['lnSize'] = np.log(df2['size'])


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize
3,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693
7,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956
8,"3PAR, Inc.",PAR,"NEW YORK STOCK EXCHANGE, INC.",7.63,4.637376e+08,0.263539,0.009174,-0.435028,0.553080,35.0,2008,19.954830
9,"3SBio, Inc. ADR",SSRX,NASDAQ,7.80,1.677846e+08,0.803987,0.051344,-0.618866,0.755128,13.0,2008,18.938192
10,"51job, Inc. ADR",JOBS,NASDAQ,6.16,1.736447e+08,1.035503,0.070602,-0.656250,1.886347,34.0,2008,18.972522


In [110]:
for i in ['Beme','eP', 'm12M', 'lnSize']:
  data_f2['z' + i] = data_f2.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))
data_f2.head()

<ipython-input-110-63e1edc3ae86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['z' + i] = data_f2.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize,zBeme,zeP,zm12M,zlnSize
3,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693,-0.164947,-0.259098,0.897826,-0.208944
7,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956,-0.855474,-0.040563,0.517404,2.242329
8,"3PAR, Inc.",PAR,"NEW YORK STOCK EXCHANGE, INC.",7.63,4.637376e+08,0.263539,0.009174,-0.435028,0.553080,35.0,2008,19.954830,-1.051584,-0.976008,0.099500,-0.844305
9,"3SBio, Inc. ADR",SSRX,NASDAQ,7.80,1.677846e+08,0.803987,0.051344,-0.618866,0.755128,13.0,2008,18.938192,1.302415,-0.160333,-0.857936,-1.470181
10,"51job, Inc. ADR",JOBS,NASDAQ,6.16,1.736447e+08,1.035503,0.070602,-0.656250,1.886347,34.0,2008,18.972522,1.298719,-0.099225,-1.158431,-1.475607


In [111]:
data_f2['score'] = data_f2['zBeme'] * coef.loc['t stat','zBeme']+ data_f2['zeP'] * coef.loc['t stat','zeP']
data_f2['rank'] = data_f2.groupby('year')['score'].apply(lambda x: pd.qcut(x, 10, labels = False))

<ipython-input-111-e0a6fa67068e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['score'] = data_f2['zBeme'] * coef.loc['t stat','zBeme']+ data_f2['zeP'] * coef.loc['t stat','zeP']
<ipython-input-111-e0a6fa67068e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['rank'] = data_f2.groupby('year')['score'].apply(lambda x: pd.qcut(x, 10, labels = False))


In [112]:
data_f2.head()

,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize,zBeme,zeP,zm12M,zlnSize,score,rank
3,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693,-0.164947,-0.259098,0.897826,-0.208944,-0.909033,4
7,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956,-0.855474,-0.040563,0.517404,2.242329,-2.110676,2
8,"3PAR, Inc.",PAR,"NEW YORK STOCK EXCHANGE, INC.",7.63,4.637376e+08,0.263539,0.009174,-0.435028,0.553080,35.0,2008,19.954830,-1.051584,-0.976008,0.099500,-0.844305,-4.445033,0
9,"3SBio, Inc. ADR",SSRX,NASDAQ,7.80,1.677846e+08,0.803987,0.051344,-0.618866,0.755128,13.0,2008,18.938192,1.302415,-0.160333,-0.857936,-1.470181,2.769646,8
10,"51job, Inc. ADR",JOBS,NASDAQ,6.16,1.736447e+08,1.035503,0.070602,-0.656250,1.886347,34.0,2008,18.972522,1.298719,-0.099225,-1.158431,-1.475607,2.882975,8


In [113]:
long = data_f2[data_f2['rank']==9]
short = data_f2[data_f2['rank']==0]

# Problem 5.6


In [114]:
ret = pd.read_csv('/content/drive/MyDrive/QIS/ret.csv')

In [115]:
ret.shape

(162315, 4)

In [116]:
ret.head()

,PERMNO,date,TICKER,RET
0,10001,20090130,EWST,0.034140
1,10001,20090227,EWST,0.056222
2,10001,20090331,EWST,-0.080537
3,10001,20090430,EWST,0.044621
4,10001,20090529,EWST,0.002941


In [117]:
ret['date']=pd.to_datetime(ret['date'].astype(str), format='%Y%m%d')
ret['year'] = ret['date'].dt.strftime('%Y')
ret['month'] = ret['date'].dt.strftime('%m')
ret['year'] = ret['year'].astype('int')
ret['month'] = ret['month'].astype('int')
ret = ret.rename(columns ={"TICKER": 'ticker'})

In [118]:
ret.head()

,PERMNO,date,ticker,RET,year,month
0,10001,2009-01-30,EWST,0.034140,2009,1
1,10001,2009-02-27,EWST,0.056222,2009,2
2,10001,2009-03-31,EWST,-0.080537,2009,3
3,10001,2009-04-30,EWST,0.044621,2009,4
4,10001,2009-05-29,EWST,0.002941,2009,5


In [168]:
li=[]
for i in range(2008, 2010):
  longRet = ret[(ret.ticker.isin(long[long.year==i].ticker))&(ret.year == i+1)]
  longRet = longRet.groupby(['year','month']).RET.mean()
  longRet = longRet.reset_index()
  longRet.columns = ['year','month','long']

  shortRet = ret[(ret.ticker.isin(short[short.year==i].ticker))&(ret.year==i+1)]
  shortRet = shortRet.groupby(['year','month']).RET.mean()
  shortRet = shortRet.reset_index()
  shortRet.columns = ['year', 'month','short']

  output = pd.merge(longRet, shortRet, left_on = ['year', 'month'], right_on =['year', 'month'] )
  li.append(output)

port = pd.concat(li, axis = 0, ignore_index = True)

port['LS'] = port['long']-port['short']
port.head()

,year,month,long,short,LS
0,2009,1,-0.063269,-0.080196,0.016927
1,2009,2,-0.150035,-0.098511,-0.051524
2,2009,3,0.129385,0.083208,0.046177
3,2009,4,0.344025,0.127671,0.216353
4,2009,5,0.132091,0.030510,0.101581


In [120]:
FF = pd.read_csv('/content/drive/MyDrive/QIS/FF.csv')
FF.head()

,dateff,mktrf,smb,hml,rf,umd
0,20050131,-0.0276,-0.0151,0.0197,0.0016,0.0321
1,20050228,0.0189,-0.0051,0.0163,0.0016,0.0316
2,20050331,-0.0197,-0.0139,0.0158,0.0021,0.0054
3,20050429,-0.0261,-0.0399,-0.0034,0.0021,-0.0085
4,20050531,0.0365,0.0289,-0.0081,0.0024,0.0043


In [121]:
FF['date']=pd.to_datetime(FF['dateff'].astype(str), format='%Y%m%d')
FF['year'] = FF['date'].dt.strftime('%Y')
FF['month'] = FF['date'].dt.strftime('%m')
FF['year'] = FF['year'].astype('int')
FF['month'] = FF['month'].astype('int')

In [122]:
FF.head()

,dateff,mktrf,smb,hml,rf,umd,date,year,month
0,20050131,-0.0276,-0.0151,0.0197,0.0016,0.0321,2005-01-31,2005,1
1,20050228,0.0189,-0.0051,0.0163,0.0016,0.0316,2005-02-28,2005,2
2,20050331,-0.0197,-0.0139,0.0158,0.0021,0.0054,2005-03-31,2005,3
3,20050429,-0.0261,-0.0399,-0.0034,0.0021,-0.0085,2005-04-29,2005,4
4,20050531,0.0365,0.0289,-0.0081,0.0024,0.0043,2005-05-31,2005,5


In [123]:
p = pd.merge(FF, port, left_on = ['year', 'month'], right_on =['year', 'month'] )
p.head()

,dateff,mktrf,smb,hml,rf,umd,date,year,month,long,short,LS
0,20090130,-0.0812,-0.0001,-0.1110,0.0000,-0.0183,2009-01-30,2009,1,-0.063033,-0.082081,0.019048
1,20090227,-0.1010,0.0016,-0.0725,0.0001,0.0418,2009-02-27,2009,2,-0.149225,-0.099023,-0.050202
2,20090331,0.0895,-0.0008,0.0352,0.0002,-0.1138,2009-03-31,2009,3,0.126316,0.079895,0.046421
3,20090430,0.1019,0.0483,0.0546,0.0001,-0.3439,2009-04-30,2009,4,0.339048,0.129415,0.209633
4,20090529,0.0521,-0.0233,-0.0021,0.0000,-0.1244,2009-05-29,2009,5,0.130916,0.031649,0.099267


In [124]:
l_mean= p['long'].mean()
s_mean= p['short'].mean()
ls_mean= p['LS'].mean()



print('long mean: '+ str(l_mean), 'short mean: '+ str(s_mean), 'LS mean: '+ str(ls_mean))

long mean: 0.04074470733610369 short mean: 0.0220664567859019 LS mean: 0.01867825055020179


In [125]:
l_n = p['long'] + 1
s_n = p['short']+ 1
ls_n = p['LS']+ 1

l_geo = stats.gmean(l_n)-1
s_geo = stats.gmean(s_n)-1
ls_geo = stats.gmean(ls_n)-1

print('long geomean: '+str(l_geo), 'short geomean: '+str(s_geo), 'ls geomean: '+str(ls_geo))

long geomean: 0.03632960102021343 short geomean: 0.020267371237921106 ls geomean: 0.01743511831148381


In [126]:
l_std= p['long'].std()
s_std= p['short'].std()
ls_std= p['LS'].std()

l_sharpe = (p['long']-p['rf'])/l_std
s_sharpe = (p['short']-p['rf'])/s_std
ls_sharpe =(p['LS']-p['rf'])/ls_std

l_sharpe = l_sharpe.mean() * np.sqrt(12)
s_sharpe = s_sharpe.mean() * np.sqrt(12)
ls_sharpe =ls_sharpe.mean() * np.sqrt(12)

print('long std: '+str(l_std), 'short std: '+str(s_std),'ls std: '+str(ls_std))
print('long sharpe ratio: '+str(l_sharpe), 'short sharpe ratio: '+str(s_sharpe), 'ls sharpe ratio: '+str(ls_sharpe))

long std: 0.09927244015349393 short std: 0.061433322703914474 ls std: 0.05303683241050243
long sharpe ratio: 1.4190198699219634 short sharpe ratio: 1.239819105693463 ls sharpe ratio: 1.2147994816575483


In [127]:
final = p
final.head()

,dateff,mktrf,smb,hml,rf,umd,date,year,month,long,short,LS
0,20090130,-0.0812,-0.0001,-0.1110,0.0000,-0.0183,2009-01-30,2009,1,-0.063033,-0.082081,0.019048
1,20090227,-0.1010,0.0016,-0.0725,0.0001,0.0418,2009-02-27,2009,2,-0.149225,-0.099023,-0.050202
2,20090331,0.0895,-0.0008,0.0352,0.0002,-0.1138,2009-03-31,2009,3,0.126316,0.079895,0.046421
3,20090430,0.1019,0.0483,0.0546,0.0001,-0.3439,2009-04-30,2009,4,0.339048,0.129415,0.209633
4,20090529,0.0521,-0.0233,-0.0021,0.0000,-0.1244,2009-05-29,2009,5,0.130916,0.031649,0.099267


In [128]:
import statsmodels.formula.api as sm # module for stats models
ols = sm.ols(formula='LS ~ mktrf', data=final).fit()
print(ols.summary()) 

                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     9.948
Date:                Tue, 28 Feb 2023   Prob (F-statistic):            0.00460
Time:                        19:39:01   Log-Likelihood:                 41.415
No. Observations:                  24   AIC:                            -78.83
Df Residuals:                      22   BIC:                            -76.47
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0093      0.010      0.959      0.3

In [129]:
import statsmodels.formula.api as sm # module for stats models
ols = sm.ols(formula='LS ~ mktrf + smb + hml', data=final).fit()
print(ols.summary()) 

                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     3.582
Date:                Tue, 28 Feb 2023   Prob (F-statistic):             0.0320
Time:                        19:39:01   Log-Likelihood:                 42.099
No. Observations:                  24   AIC:                            -76.20
Df Residuals:                      20   BIC:                            -71.49
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0133      0.011      1.233      0.2

In [130]:
import statsmodels.formula.api as sm # module for stats models
ols = sm.ols(formula='LS ~ mktrf + smb + hml + umd', data=final).fit()
print(ols.summary()) 

                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.855
Method:                 Least Squares   F-statistic:                     35.03
Date:                Tue, 28 Feb 2023   Prob (F-statistic):           1.60e-08
Time:                        19:39:01   Log-Likelihood:                 62.443
No. Observations:                  24   AIC:                            -114.9
Df Residuals:                      19   BIC:                            -109.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0010      0.005      0.212      0.8

# Problem 5.7

## Problem 5.1

In [131]:
import glob
files = glob.glob('/content/drive/MyDrive/QIS/lab5*.xlsx')
files.sort()
files
files_is = files[3:8]
files_oos = files [8:]
new_df = pd.DataFrame()
for filename in files_is:
  raw = pd.read_excel(filename, index_col = None, header =0)
  raw.columns = ['name','ticker', 'exchange', 'close', 'size', 'Beme', 'eP', 'm12M', 'yr1Ret', 'ind']
  raw['year'] = int(filename[37:41])
  new_df = pd.concat([new_df, raw], ignore_index = True)

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [132]:
new_df = new_df.dropna()
new_df.shape

(24263, 11)

In [133]:
new_df.head()

,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,21.00000,275230788.0,0.200586,-0.004286,0.670616,0.047619,43.0,2003
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,11.06000,728402398.0,0.201573,0.018987,0.128846,-0.240506,43.0,2003
3,1st Constitution Bancorp,FCCY,NASDAQ,10.28139,48373949.0,0.487561,0.061283,0.077483,0.201153,45.0,2003
4,"1st Independence Financial Group, Inc.",FIFG,NASDAQ,22.77000,27847209.0,0.764427,0.046113,0.851284,-0.152072,45.0,2003
7,1st Source Corporation,SRCE,NASDAQ,19.55455,453418606.0,0.694043,0.032079,0.374665,0.206393,45.0,2003


## Problem 5.2

In [134]:
def is_out(x):
    return abs(x - x.mean()) > x.std() * 3

In [135]:
out = new_df[['size','Beme','eP','m12M']]

#creating boolean df
outA = out.apply(is_out)

print('Outlier Boolean DF')
print(outA.head(2))
print('-------------')

print('Total Outliers')
print(outA.apply(sum))
print('-------------')
#creating boolean index
outB = outA.any(axis = 1)

print('Boolean Index')
print(outB.head(2))
print('-------------')
#remove all outliers
df = new_df[~outB]

print("Before and After Shapes")
print(new_df.shape)
print(df.shape)

#reseting index
df.reset_index(inplace = True, drop = True)
df.head(3)

Outlier Boolean DF
    size   Beme     eP   m12M
0  False  False  False  False
1  False  False  False  False
-------------
Total Outliers
size    382
Beme      1
eP       20
m12M     26
dtype: int64
-------------
Boolean Index
0    False
1    False
dtype: bool
-------------
Before and After Shapes
(24263, 11)
(23834, 11)


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,21.00000,275230788.0,0.200586,-0.004286,0.670616,0.047619,43.0,2003
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,11.06000,728402398.0,0.201573,0.018987,0.128846,-0.240506,43.0,2003
2,1st Constitution Bancorp,FCCY,NASDAQ,10.28139,48373949.0,0.487561,0.061283,0.077483,0.201153,45.0,2003


In [136]:
data_f = df[(df['eP'] >0) &(df['close'] >5)& (df['size'] > 100000000)&(df['Beme'] >0)]
data_f.shape

(14591, 11)

In [137]:
data_f['lnSize'] = np.log(df['size'])
data_f.head()

<ipython-input-137-1b1f7ca262fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f['lnSize'] = np.log(df['size'])


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,11.06000,7.284024e+08,0.201573,0.018987,0.128846,-0.240506,43.0,2003,20.406364
4,1st Source Corporation,SRCE,NASDAQ,19.55455,4.534186e+08,0.694043,0.032079,0.374665,0.206393,45.0,2003,19.932326
6,21st Century Insurance Group,TW,"NEW YORK STOCK EXCHANGE, INC.",13.75000,1.174738e+09,0.596465,0.042909,0.005902,-0.005057,34.0,2003,20.884311
10,"7-Eleven, Inc.",SE,"NEW YORK STOCK EXCHANGE, INC.",16.05000,1.790268e+09,0.189925,0.039252,1.044888,0.492212,31.0,2003,21.305631
11,99 Cents Only Stores,NDN,"NEW YORK STOCK EXCHANGE, INC.",27.23000,1.962946e+09,0.249567,0.031216,-0.009296,-0.406537,43.0,2003,21.397712


In [138]:
from scipy.stats import zscore #scipy.stats has a zscore function
import scipy.stats as stats
for i in ['Beme','eP', 'm12M', 'lnSize']:
  data_f['z' + i] = data_f.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))

<ipython-input-138-5754b203940d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f['z' + i] = data_f.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))


In [139]:
import statsmodels.formula.api as smf
coefficients = pd.DataFrame()
for i in data_f['year'].unique():
  results = smf.ols(formula = 'yr1Ret~ + zBeme+ zm12M + zeP + zlnSize', data = data_f[data_f['year'] == i]).fit()
  coeff = pd.DataFrame(results.params)
  coeff.columns = [i]
  coefficients = pd.concat([coefficients, coeff], axis = 1)

In [140]:
coef = coefficients
coef = coef.transpose()
coef.head()

,Intercept,zBeme,zm12M,zeP,zlnSize
2003,0.212352,0.018562,0.005599,0.038581,-0.004227
2004,0.100718,0.019518,0.013688,0.003552,0.019192
2005,0.186850,0.026331,-0.008218,0.007991,-0.006265
2006,0.042734,-0.026658,0.012053,-0.011895,0.025120
2007,-0.351123,-0.002493,-0.029436,-0.009038,-0.012114


##Problem 5.3

In [141]:
coef.head()

,Intercept,zBeme,zm12M,zeP,zlnSize
2003,0.212352,0.018562,0.005599,0.038581,-0.004227
2004,0.100718,0.019518,0.013688,0.003552,0.019192
2005,0.186850,0.026331,-0.008218,0.007991,-0.006265
2006,0.042734,-0.026658,0.012053,-0.011895,0.025120
2007,-0.351123,-0.002493,-0.029436,-0.009038,-0.012114


In [142]:
import math
coef.loc["FM Avg"] = coef.mean()
coef.loc["Std"] = coef.std()
coef.loc['t stat'] = (coef.loc['FM Avg',:]/coef.loc['Std',:])*math.sqrt(5)
coef.tail()

,Intercept,zBeme,zm12M,zeP,zlnSize
2006,0.042734,-0.026658,0.012053,-0.011895,0.025120
2007,-0.351123,-0.002493,-0.029436,-0.009038,-0.012114
FM Avg,0.038306,0.007052,-0.001263,0.005838,0.004341
Std,0.203925,0.019427,0.016062,0.017987,0.014893
t stat,0.420032,0.811699,-0.175813,0.725775,0.651801


## Problem 5.4

In [143]:
coef.tail()

,Intercept,zBeme,zm12M,zeP,zlnSize
2006,0.042734,-0.026658,0.012053,-0.011895,0.025120
2007,-0.351123,-0.002493,-0.029436,-0.009038,-0.012114
FM Avg,0.038306,0.007052,-0.001263,0.005838,0.004341
Std,0.203925,0.019427,0.016062,0.017987,0.014893
t stat,0.420032,0.811699,-0.175813,0.725775,0.651801


None of the following variables have a T-stat above 1.5. As such, we will take the same variables as we did in the previous exercise so that we may compare efficiently. 

## Problem 5.5

In [144]:
new_df2 = pd.DataFrame()
for filename in files_oos:
  raw = pd.read_excel(filename, index_col = None, header =0)
  raw.columns = ['name','ticker', 'exchange', 'close', 'size', 'Beme', 'eP', 'm12M', 'yr1Ret', 'ind']
  raw['year'] = int(filename[37:41])
  new_df2 = pd.concat([new_df2, raw], ignore_index = True)

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [145]:
new_df2 = new_df2.dropna()
new_df2.shape

(9433, 11)

In [146]:
out = new_df2[['size','Beme','eP','m12M']]

#creating boolean df
outA = out.apply(is_out)

print('Outlier Boolean DF')
print(outA.head(2))
print('-------------')

print('Total Outliers')
print(outA.apply(sum))
print('-------------')
#creating boolean index
outB = outA.any(axis = 1)

print('Boolean Index')
print(outB.head(2))
print('-------------')
#remove all outliers
df2 = new_df2[~outB]

print("Before and After Shapes")
print(new_df2.shape)
print(df2.shape)

#reseting index
df2.reset_index(inplace = True, drop = True)
df2.head(3)

Outlier Boolean DF
    size   Beme     eP   m12M
1  False  False  False  False
2  False  False  False  False
-------------
Total Outliers
size    146
Beme      5
eP        6
m12M     76
dtype: int64
-------------
Boolean Index
1    False
2    False
dtype: bool
-------------
Before and After Shapes
(9433, 11)
(9202, 11)


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year
0,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,3.8200,243373888.0,0.907906,0.086387,-0.643744,-0.306283,43.0,2008
1,"1st Century Bancshrs, Inc.",FCTY,NASDAQ,4.1500,40558680.0,1.406554,0.057831,-0.288136,-0.168675,45.0,2008
2,1st Constitution Bancorp,FCCY,NASDAQ,8.3093,40364929.0,1.150591,0.076970,-0.448986,-0.350248,45.0,2008


In [147]:
data_f2 = df2[(df2['eP'] >0) &(df2['close'] >5)& (df2['size'] > 100000000)&(df2['Beme'] >0)]
data_f2.shape

(4299, 11)

In [148]:
data_f2['lnSize'] = np.log(df2['size'])
data_f2.head()

<ipython-input-148-9c2ed5179e06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['lnSize'] = np.log(df2['size'])


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize
3,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693
7,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956
8,"3PAR, Inc.",PAR,"NEW YORK STOCK EXCHANGE, INC.",7.63,4.637376e+08,0.263539,0.009174,-0.435028,0.553080,35.0,2008,19.954830
9,"3SBio, Inc. ADR",SSRX,NASDAQ,7.80,1.677846e+08,0.803987,0.051344,-0.618866,0.755128,13.0,2008,18.938192
10,"51job, Inc. ADR",JOBS,NASDAQ,6.16,1.736447e+08,1.035503,0.070602,-0.656250,1.886347,34.0,2008,18.972522


In [149]:
for i in ['Beme','eP', 'm12M', 'lnSize']:
  data_f2['z' + i] = data_f2.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))
data_f2.head()

<ipython-input-149-63e1edc3ae86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['z' + i] = data_f2.groupby(['year', 'ind'])[i].apply(lambda x: stats.zscore(x))


,name,ticker,exchange,close,size,Beme,eP,m12M,yr1Ret,ind,year,lnSize,zBeme,zeP,zm12M,zlnSize
3,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693,-0.164947,-0.259098,0.897826,-0.208944
7,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956,-0.855474,-0.040563,0.517404,2.242329
8,"3PAR, Inc.",PAR,"NEW YORK STOCK EXCHANGE, INC.",7.63,4.637376e+08,0.263539,0.009174,-0.435028,0.553080,35.0,2008,19.954830,-1.051584,-0.976008,0.099500,-0.844305
9,"3SBio, Inc. ADR",SSRX,NASDAQ,7.80,1.677846e+08,0.803987,0.051344,-0.618866,0.755128,13.0,2008,18.938192,1.302415,-0.160333,-0.857936,-1.470181
10,"51job, Inc. ADR",JOBS,NASDAQ,6.16,1.736447e+08,1.035503,0.070602,-0.656250,1.886347,34.0,2008,18.972522,1.298719,-0.099225,-1.158431,-1.475607


In [150]:
data_f2['score'] = data_f2['zBeme'] * coef.loc['t stat','zBeme']+ data_f2['zeP'] * coef.loc['t stat','zeP']

<ipython-input-150-b7df7773167a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['score'] = data_f2['zBeme'] * coef.loc['t stat','zBeme']+ data_f2['zeP'] * coef.loc['t stat','zeP']


In [151]:
data_f2['rank'] = data_f2.groupby('year')['score'].apply(lambda x: pd.qcut(x, 10, labels = False))

<ipython-input-151-c26641204e55>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_f2['rank'] = data_f2.groupby('year')['score'].apply(lambda x: pd.qcut(x, 10, labels = False))


In [152]:
long = data_f2[data_f2['rank']==9]
short = data_f2[data_f2['rank']==0]

## Problem 5.6

In [153]:
ret = pd.read_csv('/content/drive/MyDrive/QIS/ret.csv')

In [154]:
ret['date']=pd.to_datetime(ret['date'].astype(str), format='%Y%m%d')
ret['year'] = ret['date'].dt.strftime('%Y')
ret['month'] = ret['date'].dt.strftime('%m')
ret['year'] = ret['year'].astype('int')
ret['month'] = ret['month'].astype('int')
ret = ret.rename(columns ={"TICKER": 'ticker'})
ret.head()

,PERMNO,date,ticker,RET,year,month
0,10001,2009-01-30,EWST,0.034140,2009,1
1,10001,2009-02-27,EWST,0.056222,2009,2
2,10001,2009-03-31,EWST,-0.080537,2009,3
3,10001,2009-04-30,EWST,0.044621,2009,4
4,10001,2009-05-29,EWST,0.002941,2009,5


In [155]:
li=[]
for i in range(2008, 2010):
  longRet = ret[(ret.ticker.isin(long[long.year==i].ticker))&(ret.year == i+1)]
  longRet = longRet.groupby(['year','month']).RET.mean()
  longRet = longRet.reset_index()
  longRet.columns = ['year','month','long']

  shortRet = ret[(ret.ticker.isin(short[short.year==i].ticker))&(ret.year==i+1)]
  shortRet = shortRet.groupby(['year','month']).RET.mean()
  shortRet = shortRet.reset_index()
  shortRet.columns = ['year', 'month','short']

  output = pd.merge(longRet, shortRet, left_on = ['year', 'month'], right_on =['year', 'month'] )
  li.append(output)

port = pd.concat(li, axis = 0, ignore_index = True)

port['LS'] = port['long']-port['short']
port.head()

,year,month,long,short,LS
0,2009,1,-0.063269,-0.080196,0.016927
1,2009,2,-0.150035,-0.098511,-0.051524
2,2009,3,0.129385,0.083208,0.046177
3,2009,4,0.344025,0.127671,0.216353
4,2009,5,0.132091,0.030510,0.101581


In [156]:
FF = pd.read_csv('/content/drive/MyDrive/QIS/FF.csv')

In [157]:
FF.shape

(168, 6)

In [158]:
FF['date']=pd.to_datetime(FF['dateff'].astype(str), format='%Y%m%d')
FF['year'] = FF['date'].dt.strftime('%Y')
FF['month'] = FF['date'].dt.strftime('%m')
FF['year'] = FF['year'].astype('int')
FF['month'] = FF['month'].astype('int')
FF.head()

,dateff,mktrf,smb,hml,rf,umd,date,year,month
0,20050131,-0.0276,-0.0151,0.0197,0.0016,0.0321,2005-01-31,2005,1
1,20050228,0.0189,-0.0051,0.0163,0.0016,0.0316,2005-02-28,2005,2
2,20050331,-0.0197,-0.0139,0.0158,0.0021,0.0054,2005-03-31,2005,3
3,20050429,-0.0261,-0.0399,-0.0034,0.0021,-0.0085,2005-04-29,2005,4
4,20050531,0.0365,0.0289,-0.0081,0.0024,0.0043,2005-05-31,2005,5


In [159]:
p = pd.merge(FF, port, left_on = ['year', 'month'], right_on =['year', 'month'] )

In [160]:
p.shape

(24, 12)

In [161]:
l_mean= p['long'].mean()
s_mean= p['short'].mean()
ls_mean= p['LS'].mean()


print('long mean: '+ str(l_mean), 'short mean: '+ str(s_mean), 'LS mean: '+ str(ls_mean))

long mean: 0.041099460923725656 short mean: 0.02224875341394901 LS mean: 0.018850707509776646


In [162]:

l_n = p['long'] + 1
s_n = p['short']+ 1
ls_n = p['LS']+ 1

l_geo = stats.gmean(l_n)-1
s_geo = stats.gmean(s_n)-1
ls_geo = stats.gmean(ls_n)-1

print('long geomean: '+str(l_geo), 'short geomean: '+str(s_geo), 'ls geomean: '+str(ls_geo))

long geomean: 0.03663887861468007 short geomean: 0.02044605247542086 ls geomean: 0.017557214217498895


In [163]:
l_std= p['long'].std()
s_std= p['short'].std()
ls_std= p['LS'].std()

l_sharpe = (p['long']-p['rf'])/l_std
s_sharpe = (p['short']-p['rf'])/s_std
ls_sharpe =(p['LS']-p['rf'])/ls_std

l_sharpe = l_sharpe.mean() * np.sqrt(12)
s_sharpe = s_sharpe.mean() * np.sqrt(12)
ls_sharpe =ls_sharpe.mean() * np.sqrt(12)

print('long std: '+str(l_std), 'short std: '+str(s_std),'ls std: '+str(ls_std))
print('long sharpe ratio: '+str(l_sharpe), 'short sharpe ratio: '+str(s_sharpe), 'ls sharpe ratio: '+str(ls_sharpe))

long std: 0.0999101926815993 short std: 0.06150739894845769 ls std: 0.054204115767855346
long sharpe ratio: 1.4222619712300344 short sharpe ratio: 1.2485928940443842 ls sharpe ratio: 1.1996602846864308


In [164]:
final = p
final.head()

,dateff,mktrf,smb,hml,rf,umd,date,year,month,long,short,LS
0,20090130,-0.0812,-0.0001,-0.1110,0.0000,-0.0183,2009-01-30,2009,1,-0.063269,-0.080196,0.016927
1,20090227,-0.1010,0.0016,-0.0725,0.0001,0.0418,2009-02-27,2009,2,-0.150035,-0.098511,-0.051524
2,20090331,0.0895,-0.0008,0.0352,0.0002,-0.1138,2009-03-31,2009,3,0.129385,0.083208,0.046177
3,20090430,0.1019,0.0483,0.0546,0.0001,-0.3439,2009-04-30,2009,4,0.344025,0.127671,0.216353
4,20090529,0.0521,-0.0233,-0.0021,0.0000,-0.1244,2009-05-29,2009,5,0.132091,0.030510,0.101581


In [165]:
import statsmodels.formula.api as sm # module for stats models
ols = sm.ols(formula='LS ~ mktrf', data=final).fit()
print(ols.summary()) 

                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     9.569
Date:                Tue, 28 Feb 2023   Prob (F-statistic):            0.00531
Time:                        19:39:24   Log-Likelihood:                 40.750
No. Observations:                  24   AIC:                            -77.50
Df Residuals:                      22   BIC:                            -75.14
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0094      0.010      0.942      0.3

In [166]:
import statsmodels.formula.api as sm # module for stats models
ols = sm.ols(formula='LS ~ mktrf + smb + hml', data=final).fit()
print(ols.summary()) 

                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     3.483
Date:                Tue, 28 Feb 2023   Prob (F-statistic):             0.0350
Time:                        19:39:24   Log-Likelihood:                 41.460
No. Observations:                  24   AIC:                            -74.92
Df Residuals:                      20   BIC:                            -70.21
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0137      0.011      1.233      0.2

In [167]:
import statsmodels.formula.api as sm # module for stats models
ols = sm.ols(formula='LS ~ mktrf + smb + hml + umd', data=final).fit()
print(ols.summary()) 

                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                     37.69
Date:                Tue, 28 Feb 2023   Prob (F-statistic):           8.70e-09
Time:                        19:39:24   Log-Likelihood:                 62.696
No. Observations:                  24   AIC:                            -115.4
Df Residuals:                      19   BIC:                            -109.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0010      0.005      0.199      0.8

## Problem 5.8

For our first backtest window, from 2000 to 2007 we got the following regression results:
- CAPM - Adj R^2 = 28%
- F3 - Adj R^2 = 25.2%
- Carhart4 - Adj R^2 = 85.5%

For our first backtest window, from 2003 to 2007 we got the following regression results:
- CAPM - Adj R^2 = 27.1%
- F3 - Adj R^2 = 24.5%
- Carhart4 - Adj R^2 = 86.5%

Apart from the Carhart 4 factor model which performed better in the second backtest window, our backtest was better in the first window for the CAPM and F3 model.
